Books Recommendation System using Clustering | Collaborative based

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print(pd.__version__)

2.2.1


Loading Three Datasets Books,Users and rating information

1.We are loading each datasets
2.Renaming column name

In [3]:
books=pd.read_csv('Data/BX-Books.csv',on_bad_lines='skip',sep=";",encoding='latin-1')

C:\Users\DELL 3567\AppData\Local\Temp\ipykernel_10424\2855233194.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Data/BX-Books.csv',on_bad_lines='skip',sep=";",encoding='latin-1')


In [4]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [5]:
books.shape

(271360, 8)

In [6]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [7]:
books=books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [8]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [9]:
books.rename(columns={
    "Book-Title":"title",
    "Book-Author":"author",
     "Year-Of-Publication":"year",
     "Publisher":"publisher",
    "Image-URL-L":"img_url"
     },inplace=True)

In [10]:
books.head(2)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [11]:
users=pd.read_csv('Data/BX-Users.csv',on_bad_lines='skip',sep=";",encoding='latin-1')

In [12]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [13]:
users.shape

(278858, 3)

In [14]:
ratings=pd.read_csv('Data/BX-Book-Ratings.csv',on_bad_lines='skip',sep=";",encoding='latin-1')

In [15]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [16]:
ratings.shape

(1149780, 3)

In [17]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [18]:
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [19]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [20]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [21]:
ratings['user_id'].unique().shape

(105283,)

In [22]:
x=ratings['user_id'].value_counts() > 200
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [23]:
x[x].shape

(899,)

In [24]:
y=x[x].index

In [25]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [26]:
ratings=ratings[ratings['user_id'].isin(y)]

In [27]:
ratings

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [28]:
ratings.shape

(526356, 3)

In [29]:
ratings_with_books=ratings.merge(books,on="ISBN")

In [30]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


In [31]:
ratings_with_books.shape

(487671, 8)

In [32]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [33]:
num_rating.head(10)

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
5,Dark Justice,1
6,Deceived,1
7,Earth Prayers From around the World: 365 Pray...,3
8,Final Fantasy Anthology: Official Strategy Gu...,3
9,Flight of Fancy: American Heiresses (Zebra Ba...,1


In [34]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [35]:
final_rating=ratings_with_books.merge(num_rating,on='title')

In [36]:
final_rating.head(5)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13


In [37]:
final_rating.shape

(487671, 9)

In [38]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [39]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
73440,36606,0684848783,0,Tis : A Memoir,Frank McCourt,1999,Scribner,http://images.amazon.com/images/P/0684848783.0...,53
72658,36606,0375705856,0,Plainsong (Vintage Contemporaries),KENT HARUF,2000,Vintage,http://images.amazon.com/images/P/0375705856.0...,62
412505,234828,0345354621,0,The Terminal Man,Michael Crichton,1988,Ballantine Books,http://images.amazon.com/images/P/0345354621.0...,50
297239,172742,0670839809,0,Waiting to Exhale,Terry McMillan,1992,Viking Books,http://images.amazon.com/images/P/0670839809.0...,68
472389,265889,0553569910,0,The Ugly Duckling,Iris Johansen,1997,Bantam Books,http://images.amazon.com/images/P/0553569910.0...,65
238385,136205,0553292722,0,Still Waters,TAMI HOAG,1992,Bantam,http://images.amazon.com/images/P/0553292722.0...,83
338698,197775,0671894455,10,FORREST GUMP (Movie Tie in),Winston Groom,1994,Pocket,http://images.amazon.com/images/P/0671894455.0...,70
423529,238545,044651652X,0,The Bridges of Madison County,Robert James Waller,1992,Warner Books,http://images.amazon.com/images/P/044651652X.0...,161
427970,240567,0553106341,2,Dust to Dust,Tami Hoag,2000,Bantam Books,http://images.amazon.com/images/P/0553106341.0...,78
176471,101209,0440215625,0,Dragonfly in Amber,DIANA GABALDON,1993,Dell,http://images.amazon.com/images/P/0440215625.0...,71


In [40]:
final_rating.drop_duplicates({"user_id","title"},inplace=True)

In [41]:
final_rating.shape

(59850, 9)

In [42]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [43]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [45]:
book_pivot.fillna(0,inplace=True)

In [46]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
 from scipy.sparse import csc_matrix

In [53]:
book_sparse=csc_matrix(book_pivot)

In [54]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14961 stored elements in Compressed Sparse Column format>

In [55]:
from sklearn.neighbors import NearestNeighbors

In [56]:
model=NearestNeighbors(algorithm='brute')

In [57]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [61]:
distance,suggestion=model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=10)

In [62]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143, 76.32823855, 76.62245102, 76.64202503, 76.75285011]])

In [63]:
suggestion

array([[237, 238, 240, 241, 184, 536, 291, 686, 675, 689]], dtype=int64)

In [65]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall', 'Jacob Have I Loved',
       'Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))',
       'The Witness', 'Toxin'],
      dtype='object', name='title')


In [67]:
book_pivot.index[10]

'A Darkness More Than Night'

In [68]:
books_name=book_pivot.index

In [69]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [70]:
import pickle

In [71]:

pickle.dump(model,open('artifacts/model.pkl','wb'))

pickle.dump(books_name,open('artifacts/book_name.pkl','wb'))

pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))

pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))


Test Our Recommendation System How it works

In [72]:
np.where(book_pivot.index=='2nd Chance')[0]
    

In [77]:
np.where(book_pivot.index=='2nd Chance')[0][0]

2

In [92]:
def recommned_book(book_name):
    book_id=np.where(book_pivot.index==book_name)[0][0]
    distance,suggestion=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=10)
    
    for i in range(len(suggestion)):
        books=book_pivot.index[suggestion[i]]
        #print(books)
        for j in books:
            print(j,end=" | ")

In [95]:
book_name='Harry Potter and the Chamber of Secrets (Book 2)'
recommned_book(book_name)

Harry Potter and the Chamber of Secrets (Book 2) | Harry Potter and the Goblet of Fire (Book 4) | Harry Potter and the Prisoner of Azkaban (Book 3) | Harry Potter and the Sorcerer's Stone (Book 1) | Exclusive | The Cradle Will Fall | Jacob Have I Loved | Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback)) | The Witness | Toxin | 

In [96]:
import sys

In [97]:
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
